## Import

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import pandas as pd
from pathlib import Path
import os
from tqdm import tqdm
import numpy as np

from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# for .dcm file
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

## Load submission file

If submission file is public, just submit fast!

In [ ]:
submission = pd.read_csv("../input/siim-covid19-detection/sample_submission.csv")
if submission.shape[0] == 2477:
    fast_sub = True
else:
    fast_sub = False

## .dcm to .jpg

In [ ]:
# extract image pixel data from .dcm
def read_xray(path):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
# convert .dcm to .jpg
os.makedirs('/kaggle/tmp/siim/images/test', exist_ok=True)
for path in tqdm(Path("../input/siim-covid19-detection/test").rglob('*.dcm')):
    xray = read_xray(str(path))
    im = Image.fromarray(xray)
    im.save(os.path.join('/kaggle/tmp/siim/images/test', path.name.replace('.dcm', '.jpg')))
    if fast_sub: break

## Inference YOLOv5

In [ ]:
# Copy YOLOv5 directory to working directory
!cp -r /kaggle/input/yolov5/yolov5 yolov5

In [ ]:
os.chdir('/kaggle/working/yolov5')
!rm -rf runs/detect/
!rm -rf runs/test/
!python3 detect.py \
        --weights ../input/myweight/best.pt \
        --source /kaggle/tmp/siim/images/test \
        --iou-thres 0.5 \
        --save-txt \
        --save-conf \
        --device 0
os.chdir('/kaggle/working')